# Notebook to look at linearity

Initially written 31 Aug 2020 by Craig Lage.\
Using pkl'd data so I can run it on my laptop.


In [ ]:
import sys, os, glob, time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
from scipy.optimize import fmin_powell
import scipy.interpolate as interp
import astropy.io.fits as pf

In [ ]:
DATA_DIR = '/project/shared/BOT/'
REPO_DIR = '/project/cslage/BOT_lspdev/E2V_6790D_Gain_Mu12_R22S11/'
RUN = '6790D'
[RAFT, SENSOR, DETECTOR] = ['R22', 'S11', 94]
#[RAFT, SENSOR, DETECTOR] = ['R02', 'S02', 11]

In [ ]:
file = open('/Users/cslage/Research/LSST/code/BOT/gain/linearized/pickle/linearity_r22s11_c04.pkl','rb')
#file = open('/project/cslage/BOT_lspdev/ptcs/linearity_r22s11_c04.pkl','rb')
pkl_data = pkl.load(file)
file.close()
print(pkl_data.keys())
rawMeans = pkl_data['rawMeans']
bothRawMeans = pkl_data['bothRawMeans']
headerData = pkl_data['headerData']
rawVars = pkl_data['rawVars']
means = pkl_data['means']
expTimes = pkl_data['expTimes']
gains = pkl_data['gains']
pairs = pkl_data['pairs']

In [ ]:
flux_max_e = 150000  # electrons
monDiodeNormalizer = 3.0E-9
#pdf = PdfPages(REPO_DIR+'plots/Flux_Linearity_FF_%s%s.pdf'%(RAFT,SENSOR))
for amp in ['C04']:#rawMeans.keys():
    smallest_flux = 100000.0
    flux_max = flux_max_e / gains[amp] # In ADU
    found_sat = False
    fig = plt.figure(figsize=(16,8))
    plt.suptitle("Flux linearity; Run%s; %s%s; Amp %s"%(RUN,RAFT,SENSOR,amp),fontsize=24)
    ax1 = plt.subplot(1,2,1)
    ax2 = plt.subplot(1,2,2)
    all_x = []
    all_y = []
    fit_x = []
    fit_y = []
    filters = []
    
    for myFilter2 in ['1.0','0.5','0.3','0.01']:
        yaxis = []
        xaxis = []
        all_filters = []
        for i, visits in enumerate(pairs):
            test = (visits[0] in headerData.keys()) and (visits[1] in headerData.keys())
            if not test:
                continue
            for j, visit in enumerate(visits):
                flux = bothRawMeans[amp][i][j]
                mon = headerData[visit][2] / monDiodeNormalizer
                exptime = headerData[visit][0]
                filter2 = headerData[visit][1].split('D')[2]
                if filter2 == '0.01' and flux < smallest_flux:
                    smallest_flux = flux # Smallest flux for ND_D0.01 filter
                    x1_max = exptime * mon * 2.0
                if filter2 == '0.01' and flux > flux_max and not found_sat:
                    x2_max = exptime * mon * 2.0
                    found_sat = True
                if filter2 == myFilter2:
                    xaxis.append(exptime * mon)
                    yaxis.append(flux)
                    all_filters.append(filter2)
                    if flux < flux_max:
                        all_x.append(exptime * mon)
                        all_y.append(flux)
                        fit_x.append(exptime * mon)
                        fit_y.append(flux)
                        filters.append(filter2)
                        
        ax1.scatter(xaxis, yaxis, label = 'FILTER2=ND_0D%s'%myFilter2)
        ax2.scatter(xaxis, yaxis, label = 'FILTER2=ND_0D%s'%myFilter2)
    xaxis = np.array(xaxis)
    all_x = np.array(all_x)
    all_y = np.array(all_y)
    fit_x = np.array(fit_x)
    fit_y = np.array(fit_y)
    xplot = np.linspace(0,x2_max, 500)
    linear_fit = np.polyfit(fit_x, fit_y, 1)
    print(linear_fit)

    lin_yplot = xplot * linear_fit[0]# + linear_fit[1]  
    ax1.plot(xplot, lin_yplot, color='black', ls = '-', label = 'Linear')
    ax1.set_xlim(0,x1_max)
    ax1.set_ylim(0,smallest_flux * 2.0)
    ax1.set_xlabel('EXPTIME * MONDIODE', fontsize=18)
    ax1.set_ylabel('Flux(ADU)-From PTC rawMeans', fontsize=18)
    ax1.legend(loc='lower right', fontsize=12)

    ax2.plot(xplot, lin_yplot, color='black', ls = '-', label = 'Linear')
    ax2.set_xlim(0,x2_max)
    ax2.set_ylim(0,flux_max*2.0)
    ax2.set_xlabel('EXPTIME * MONDIODE', fontsize=18)
    ax2.set_ylabel('Flux(ADU)-From PTC rawMeans', fontsize=18)
    ax2.legend(loc='lower right', fontsize=12)

    #pdf.savefig(fig)  # saves the current figure into a pdf page
    #plt.close()
#pdf.close()


In [ ]:
index = expTimes.argsort()
means = means[index]
expTimes = expTimes[index]

lin_index = np.where(means < 20000)
coefs = np.polyfit(means[lin_index], expTimes[lin_index], 1)
slope = coefs[0]

lin_model = slope * means

lin_residuals = (expTimes - lin_model) / expTimes * 100.0

xplot = np.linspace(100.0,flux_max, 200)
yplot = slope * xplot

knots = 10
xx = means
yy = expTimes

# Force the spline to match the linear fit at low flux by adding 
# highly weighted points at low flux
xx_points = np.array([0.0,500.0,5000.0,20000.0])
yy_points = xx_points * slope
xx = np.insert(xx, 0, xx_points)
yy = np.insert(yy, 0, yy_points)
weights = np.ones_like(xx)
weights[0:len(xx_points)] = 100000.0

index = xx. argsort()
xx = xx[index]
yy = yy[index]
weights = weights[index]

length = xx[-1]-xx[0]
t = np.linspace(xx[0]+1e-5*length, xx[-1]-1e-5*length, knots)
print("Input knots",t[1:-2])
s = interp.splrep(xx, yy, w=weights, task=-1, t=t[1:-2])
print("Output knots",s[0])
s_model = interp.splev(means, s) 
s_yplot = interp.splev(xplot, s)
s_residuals = (expTimes - s_model) / expTimes * 100.0

fig = plt.figure(figsize = (16,16))
plt.suptitle("Flux Nonlinearity - Run 6790D, R22S11 C04 28Aug2020", fontsize = 24)
plt.subplot(2,1,1)
plt.plot(xplot, xplot/yplot*slope, color='green', label = 'Low flux linear fit')
plt.plot(xplot, xplot/s_yplot*slope, color='blue', label = '%d knot spline fit'%knots)
#plt.scatter(means, means/expTimes, marker = 'x', s=100, color='red')
plt.scatter(xx, xx/yy*slope, marker = 'x', s=100, color='red')
plt.ylabel("Flux/ExpTime*LowFluxSlope", fontsize = 18)
plt.xlabel("Flux(ADU)", fontsize = 18)
plt.legend(fontsize=18)
#plt.ylim(5320,5360)
#plt.xscale('log')

plt.subplot(2,1,2)
plt.scatter(means[1:-1],lin_residuals[1:-1], label = 'Linear Residuals')
plt.scatter(means[1:-1],s_residuals[1:-1], label = 'Spline Residuals')
plt.plot([0.0, 120000.0], [0.0,0.0], ls = '--', color = 'black')
plt.ylabel("Residuals (%)", fontsize = 18)
plt.xlabel("Flux(ADU)", fontsize = 18)

#plt.xscale('log')
plt.legend(fontsize = 18)
#plt.savefig(REPO_DIR + 'plots/Flux_Correction_R22S11_C04_28Aug20.pdf')

In [ ]:
interp.splev(0, s)

In [ ]:
amp = 'C04'
rvars = np.array(rawVars[amp])
rmeans = np.array(rawMeans[amp])
correctedMeans = rmeans + interp.splev(rmeans, s)
fig = plt.figure(figsize = (16,8))
plt.title("Non-linearity, R22S11, C04", fontsize = 24)
cut_means = []
cut_vars = []
# Fit a line to the points outside the bump
for i, mean in enumerate(rmeans):
    if (mean >10000 and mean <25000) or (mean >65000 and mean <100000):
        cut_means.append(mean)
        cut_vars.append(rvars[i]/mean)
xplot = np.linspace(-5000,120000, 100)
these_coefs = np.polyfit(cut_means, cut_vars, 1)
yplot = these_coefs[1] + these_coefs[0] * xplot
plt.scatter(rmeans,rvars/rmeans, color = 'red', marker = 'o', label = 'UnCorrected')
plt.scatter(correctedMeans,rvars/correctedMeans, color = 'green', marker = 'x', s = 100,label = 'Corrected')
plt.plot(xplot, yplot, ls = '--', color = 'blue')
plt.xlabel("Flux(ADU)", fontsize = 18)
plt.ylabel("Variance / Flux (ADU)", fontsize = 18)
plt.ylim(0.6, 1.1)
plt.legend(fontsize = 18)
#plt.savefig(REPO_DIR + 'plots/NonLinearity_R22S11_C04_28Aug20.pdf')